In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

def fft(signal):
    """discrete fourier transform of the input"""
    N = len(signal)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)
    return np.dot(e, signal)

def fftfreq(n, d=1):
    """computing the sample frequencies for the discrete fourier transform"""
    val = 1.0 / (n * d)
    N = (n - 1) // 2 + 1
    results = np.arange(0, N, dtype=int)
    return np.concatenate((results, results - N)) * val

def ifft(signal):
    """computing the inverse discrete fourier transform of the input"""
    N = len(signal)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(2j * np.pi * k * n / N)
    return np.dot(e, signal) / N

In [2]:
def butterworth(freq, cutoff_freq, order):
    """
    compute the Butterworth filter frequency response

    parameters:
    - freq: array - the frequency
    - cutoff_freq: float - the cutoff frequency of the filter.
    - order: int - the order of the Butterworth filter.

    returns:
    - array: - the Butterworth filter frequency response
    """
    return 1 / np.sqrt(1 + (freq / cutoff_freq)**(2 * order))

def apply_butterworth(signal, cutoff_freq, order, sampling_rate):
    """
    apply the Butterworth filter

    parameters:
    - signal: array - The input signal
    - cutoff_freq: float - the cutoff frequency of the filter
    - order: int - the order of the Butterworth filter
    - sampling_rate: float - the sampling rate of the input signal

    return:
    - array: - the filtered signal
    """
    freq_signal = fft(signal)
    freqs = fftfreq(len(signal), d=1/sampling_rate)

    filter_response = butterworth(freqs, cutoff_freq, order)

    filtered_freq_signal = freq_signal * filter_response
    return ifft(filtered_freq_signal).real

In [ ]:
def interactive_butterworth(cutoff_freq=50, order=2):
    filtered_signal = apply_butterworth(signal, cutoff_freq, order, sampling_rate)
    plt.figure(figsize=(10, 6))
    plt.plot(t, signal, label='Original Signal')
    plt.plot(t, filtered_signal, label='Filtered Signal')
    plt.xlabel('time')
    plt.ylabel('amplitude')
    plt.title('own Butterworth filter')
    plt.legend()
    plt.grid(True)
    plt.show()

sampling_rate = 1000
signal_duration = 1
t = np.linspace(0, signal_duration, sampling_rate * signal_duration, endpoint=False)
signal = np.sin(2 * np.pi * 10 * t) + np.sin(2 * np.pi * 100 * t)

cutoff_slider = FloatSlider(min=1, max=100, step=1, value=50, description='cutoff (hz)')
order_slider = IntSlider(min=1, max=10, step=1, value=2, description='order')

interact(interactive_butterworth, cutoff_freq=cutoff_slider, order=order_slider)

interactive(children=(FloatSlider(value=50.0, description='cutoff (hz)', min=1.0, step=1.0), IntSlider(value=2…